In [1]:
import os
import random
import math
import torch
import pickle
import torch.utils.data as data
import pandas as pd
from PIL import Image
import numpy as np
import cv2
import json

#### 骨头、关键点、角度的转换

In [2]:
def angle2type_V2(alpha,beta):
    if alpha >=60:
        if beta<=55:
            return 'Ia'
        else:
            return 'Ib'
    elif alpha >=50:
        return 'IIa'
    elif alpha >= 43 :
        if beta <= 77:       
            return 'IIc'
        else:
            return 'D'
    else :
        return 'III'

def angle2type_V1(alpha,beta):
    if alpha >=60:
        if beta<=55:
            return 'I'
        else:
            return 'I'
    elif alpha >=50:
        return 'II'
    elif alpha >= 43 :
        if beta <= 77:       
            return 'II'
        else:
            return 'D'
    else :
        return 'III'

def mask2point(mask):
    point_list = np.zeros((6,2))
    # 获取1号点，用2号区域的最左端代替
    if np.sum(mask==2) > 0:
        index = np.argwhere(mask==2)
    else:
        index = np.argwhere(mask==0)
    y = np.argmin(index[:,1])
    point_list[1-1] = index[y]
    # 获取2号点，用1号点平移代替
    point_list[2-1][0] = point_list[1-1][0].item()
    point_list[2-1][1] = point_list[1-1][1].item()+20
    # 获取3号点，用1号区域的最右端代替
    if np.sum(mask==1) > 0:
        index = np.argwhere(mask==1)
    else:
        index = np.argwhere(mask==0)
    y = np.argmax(index[:,1])
    point_list[3-1] = index[y]
    
    # 获取4号点，位于区域3最下端和最右端的平均
    if np.sum(mask==3) > 0:
        index = np.argwhere(mask==3)
    else:
        index = np.argwhere(mask==0)
    y1 = np.argmax(index[:,0])
    y2 = np.argmax(index[:,1])
    point_list[4-1] = np.round(((index[y1]+index[y2])/2))
    '''
    # 获取4号点，位于区域3最右端
    if np.sum(mask==3) > 0:
        index = np.argwhere(mask==3)
    else:
        index = np.argwhere(mask==0)
    y2 = np.argmax(index[:,1])
    point_list[4-1] = index[y2]
    '''
    # 获取5号点，位于区域1最下端
    if np.sum(mask==1) > 0:
        index = np.argwhere(mask==1)
    else:
        index = np.argwhere(mask==0)
    index = np.argwhere(mask==1)
    y = np.argmax(index[:,0])
    point_list[5-1] = index[y]
    # 获取6号点，位于区域4几何中心
    if np.sum(mask==4) > 0:
        index = np.argwhere(mask==4)
    else:
        index = np.argwhere(mask==0)
    p = np.mean(index, axis=0)
    point_list[6-1] = np.round(p)
    return point_list

def point2line(point1,point2):
    # 用极坐标 r,rho 来表示过两点的直线

    # 提取坐标值
    x1, y1 = point1
    x2, y2 = point2

    # 计算极径（r）原点到直线的距离
    A = y2 - y1
    B = x1 - x2
    C = x2 * y1 - x1 * y2
    # 计算原点到直线的距离（点到直线的距离公式）
    r = np.abs(C) / np.sqrt(A**2 + B**2)

    # direction 切点位于y轴的上方还是下方，也就是直线的截距
    direction = (y1*x2-y2*x1)*(x2-x1)
    if direction >=0:
        direction = 1
    else:
        direction = -1

    r*= direction

    # 计算极角（theta）（以弧度为单位）
    theta = np.arctan2(y2 - y1, x2 - x1)+np.pi/2    # 切线和直线垂直
    # +pi/2是把极轴从x轴转为y轴

    # 将极角从弧度转换为度
    theta_degrees = np.degrees(theta)

    return r,theta_degrees  # 其实是极坐标表示的切点位置，从y轴顺时针旋转

def points2angle(points):
    points = points.reshape(3,2,2)
    line_list = [point2line(ps[0], ps[1]) for ps in points ]
    theta_list = [line[1] for line in line_list ]
    alpha,beta = line2angle(*tuple(theta_list))
    return [alpha,beta]

def line2angle(theta1,theta2,theta3):
    # 这个theta是直线的切点的角度，相当于从上y轴开始顺时针
    theta1 = 90
    alpha = theta2-theta1
    beta = theta1-theta3
    return alpha,beta

#### loss 指标

In [3]:
def IOU_metric(pred,label, class_num):
    '''
        pred, label : shape(H,W) , number 0~4
    '''
    IOU_list = []
    for i in range(1,class_num):
        index_pred = (pred==i)
        index_label = (label==i)
        IOU_list.append( (index_pred&index_label).sum() / ((index_pred|index_pred).sum()+1e-5) )
    return IOU_list

def DSC_metric(pred,label, class_num):
    '''
        pred, label : shape(H,W) , number 0~4
    '''
    DSC_list = []
    for i in range(1,class_num):
        index_pred = (pred==i)
        index_label = (label==i)
        DSC_list.append( 2*(index_pred&index_label).sum() / (index_pred.sum()+1e-5+index_label.sum()) )
    return DSC_list

#### 文件读取

In [4]:

#model = 'hrnet_w48'
#pred_dir = f'/home/hln/Segmentation/mmpose/HipJoint/work_dirs/{model}/pred.pkl'

def load_result(pred_dir):
    with open(pred_dir,'rb') as f:
        result = pickle.load(f)
    pred = []
    label = []
    image = []
    for res in result:
        pred.append(res['pred_instances']['keypoints'])
        label.append(res['gt_instances']['keypoints'])
        image.append(res['img_path'])
    return pred,label,image

#### 评估性能

In [5]:
def keypoint_evalute(pred_dir):
    result = {
        'PE6':[],
        'PE4':[],
        'SDR1':[],
        'SDR2':[],
        'SDR3':[],
        'SDR4':[],
        'alpha':[],
        'beta':[],
        'typeV1':[],
        'typeV2':[]
    }

    pred_list, label_list, image_list = load_result(pred_dir)

    for id in range(len(pred_list)):
        pred_point = pred_list[id].squeeze(0)
        label_point = label_list[id].squeeze(0)

        #pred_point = pred_point[:,[1,0]]
        #label_point = label_point[:,[1,0]]
        PE=np.sqrt(((pred_point-label_point)**2).sum(-1))[2:]*0.1
        result['PE4'].append(PE.mean())
        result['SDR1'].append((PE<0.5).mean())
        result['SDR2'].append((PE<1).mean())
        result['SDR3'].append((PE<1.5).mean())
        result['SDR4'].append((PE<2).mean())

        result['PE6'].append(np.sqrt(((pred_point-label_point)**2).sum(-1)).mean()*0.1)

        label_angle = points2angle(label_point)
        pred_angle = points2angle(pred_point)

        alpha_error = abs(pred_angle[0]-label_angle[0])
        beta_error = abs(pred_angle[1]-label_angle[1])
        result['alpha'].append(alpha_error)
        result['beta'].append(beta_error)

        typeV1 = angle2type_V1(*tuple(label_angle)) == angle2type_V1(*tuple(pred_angle))
        typeV2 = angle2type_V2(*tuple(label_angle)) == angle2type_V2(*tuple(pred_angle))
        result['typeV1'].append(typeV1)
        result['typeV2'].append(typeV2)

    for key,value in result.items():
        result[key] = np.array(value)

    return result
#model = 'hrnet_w48'
#pred_dir = f'/home/hln/Segmentation/mmpose/HipJoint/work_dirs/{model}/pred.pkl'
#keypoint_evalute(pred_dir)

#### 保存结果

In [12]:
import pandas as pd

def save_result(result,save_dir):
    # 数据
    data = {}
    data['PE6平均']=[round(result['PE6'].mean(),4)]
    data['PE4平均']=[round(result['PE4'].mean(),4)]
    data['SDR1']=[round(result['SDR1'].mean(),4)]
    data['SDR2']=[round(result['SDR2'].mean(),4)]
    data['SDR3']=[round(result['SDR3'].mean(),4)]
    data['SDR4']=[round(result['SDR4'].mean(),4)]
    data['alpha']=[round(result['alpha'].mean(),2)]
    data['beta']=[round(result['beta'].mean(),2)]
    data['typeV1']=[round(result['typeV1'].mean(),4)]
    data['typeV2']=[round(result['typeV2'].mean(),4)]

    # 创建DataFrame
    df = pd.DataFrame(data)

    # 保存为Excel文件
    file_path = save_dir+'.xlsx'
    df.to_excel(file_path, index=False)

#### main

In [13]:
#model_list = ['hrnet_w48','hourglass','pvtv2','simple','hrformer']
model_list = ['darkpose','pvt','swin','SCNet']

for model in model_list:
    pred_dir = f'/home/huliangni/Segmentation/mmpose/HipJoint/work_dirs/{model}/pred.pkl'
    save_dir = f'/home/huliangni/Segmentation/mmpose/HipJoint/saved/{model}'

    result = keypoint_evalute(pred_dir)
    save_result(result, save_dir)

    print(f"平均点误差PE4 {result['PE4'].mean()},  平均点误差PE6 {result['PE6'].mean()}")
    print(f"alpha 角度误差 {result['alpha'].mean()}, beta 角度误差 {result['beta'].mean()}" )
    print(f"typeV1疾病准确率 {result['typeV1'].mean()}, typeV2疾病准确 {result['typeV2'].mean()}" )

平均点误差PE4 0.7975144386291504,  平均点误差PE6 0.9290474736007157
alpha 角度误差 2.968677684284524, beta 角度误差 3.8081919402554427
typeV1疾病准确率 0.8789808917197452, typeV2疾病准确 0.8089171974522293
平均点误差PE4 0.8073144555091858,  平均点误差PE6 0.8935285138476426
alpha 角度误差 2.224383334806896, beta 角度误差 3.9816184263375622
typeV1疾病准确率 0.8980891719745223, typeV2疾病准确 0.8535031847133758
平均点误差PE4 0.9953820109367371,  平均点误差PE6 1.0468051643128606
alpha 角度误差 2.9143597021939147, beta 角度误差 4.400564888088835
typeV1疾病准确率 0.89171974522293, typeV2疾病准确 0.7834394904458599
平均点误差PE4 0.8303765058517456,  平均点误差PE6 0.9355434408613071
alpha 角度误差 3.8780172469190153, beta 角度误差 4.047612083885537
typeV1疾病准确率 0.8726114649681529, typeV2疾病准确 0.7898089171974523


In [11]:
result['SDR2']

array([1.  , 0.25, 0.75, 0.75, 0.75, 1.  , 0.75, 0.5 , 1.  , 0.25, 0.75,
       0.75, 0.75, 1.  , 1.  , 0.75, 0.25, 0.5 , 1.  , 0.75, 0.75, 0.5 ,
       0.75, 1.  , 0.5 , 1.  , 0.75, 0.  , 1.  , 1.  , 0.5 , 0.5 , 1.  ,
       1.  , 0.75, 0.75, 1.  , 0.5 , 0.75, 1.  , 1.  , 1.  , 0.5 , 1.  ,
       0.75, 0.5 , 1.  , 0.25, 0.75, 0.75, 1.  , 1.  , 0.75, 1.  , 1.  ,
       0.75, 1.  , 0.  , 1.  , 1.  , 1.  , 0.75, 0.75, 1.  , 0.75, 1.  ,
       0.5 , 1.  , 0.75, 0.75, 0.75, 0.5 , 0.75, 1.  , 1.  , 0.75, 1.  ,
       1.  , 1.  , 1.  , 0.75, 0.75, 1.  , 0.75, 1.  , 0.75, 0.5 , 1.  ,
       1.  , 1.  , 0.75, 0.75, 1.  , 0.75, 1.  , 0.5 , 0.75, 0.5 , 0.75,
       0.75, 1.  , 0.75, 0.75, 0.25, 0.5 , 0.75, 1.  , 0.75, 1.  , 1.  ,
       1.  , 1.  , 0.5 , 0.75, 1.  , 0.75, 0.5 , 0.75, 0.75, 1.  , 0.75,
       0.75, 1.  , 1.  , 1.  , 0.75, 0.75, 0.5 , 0.5 , 0.5 , 0.75, 1.  ,
       0.75, 1.  , 1.  , 0.75, 1.  , 0.5 , 1.  , 0.5 , 0.75, 0.75, 0.75,
       0.75, 0.25, 0.75, 1.  , 0.75, 1.  , 0.75, 1.